In [1]:
import numpy as np 
import math

In [2]:
def relu(X):
    return np.maximum(X, 0)

In [3]:
def build_model(X , hidden_nodes , output_dim=2):
    model = {} 
    input_dim = X.shape[1]
    
    model['W1'] = np.random.randn(input_dim, hidden_nodes) / np.sqrt(input_dim)
    model['b1'] = np.zeros((1, hidden_nodes))
    model['W2'] = np.random.randn(hidden_nodes, output_dim) / np.sqrt(hidden_nodes)
    model['b2'] = np.zeros((1, output_dim))
    
    return model

In [4]:
def feed_forward(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    z1 = x.dot(W1) + b1
    a1 = relu(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    out = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return z1, a1, z2, out

In [5]:
def calculate_loss(model,X,y,reg_lambda):
    num_examples = X.shape[0]
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    
    z1, a1, z2, out = feed_forward(model, X)
    probs = out / np.sum(out, axis=1, keepdims=True)
    
    corect_logprobs = -np.log(probs[range(num_examples), y])
    loss = np.sum(corect_logprobs)
    
    # Add regulatization term to loss (optional)
    loss = loss + reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_examples * loss

In [6]:
def relu_derivative(X):
    return 1. * (X > 0)

In [7]:
def backprop(X,y,model,z1,a1,z2,output,reg_lambda):
    delta3 = output
    delta3[range(X.shape[0]), y] -= 1      #yhat - y
    dW2 = (a1.T).dot(delta3)
    db2 = np.sum(delta3, axis=0, keepdims=True)
    
    delta2 = delta3.dot(model['W2'].T) * relu_derivative(a1) 
    dW1 = np.dot(X.T, delta2)
    db1 = np.sum(delta2, axis=0)
   
    # Add regularization terms
    dW2 = dW2 + reg_lambda * model['W2']
    dW1 = dW1 + reg_lambda * model['W1']
    return dW1, dW2, db1, db2

In [8]:
def train(model, X, y, num_passes=10000, reg_lambda = .1, learning_rate=0.1):
    # Batch gradient descent
    done = False
    previous_loss = float('inf')
    i = 0
    losses = []
    
    while done == False:  
        z1,a1,z2,output = feed_forward(model, X)
        dW1, dW2, db1, db2 = backprop(X,y,model,z1,a1,z2,output,reg_lambda)
        
        model['W1'] -= learning_rate * dW1
        model['b1'] -= learning_rate * db1
        model['W2'] -= learning_rate * dW2
        model['b2'] -= learning_rate * db2
        
        if i % 1000 == 0:
            loss = calculate_loss(model, X, y, reg_lambda)
            losses.append(loss)
            print( "Loss after iteration %i: %f" %(i, loss)  )
            if (previous_loss-loss)/previous_loss < 0.01:
                   done = True
                    #print i
            previous_loss = loss
        i += 1
    return model, losses

In [9]:
X = np.array( [[ 1 , 1], [ 2 , 2],[ 7 , 7], [ 8 , 8], [ 9 , 9]] )

y = np.array([0,0,1,1,1])

In [10]:
model = build_model(X , 3 , 2)

In [11]:
model, losses = train ( model, X , y , reg_lambda = 0.01, learning_rate = 0.01 )

Loss after iteration 0: 0.954058
Loss after iteration 1000: 0.019403
Loss after iteration 2000: 0.014625


C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_11748/3779028765.py:21: RuntimeWarning: invalid value encountered in double_scalars
  if (previous_loss-loss)/previous_loss < 0.01:


Loss after iteration 3000: 0.012581
Loss after iteration 4000: 0.011155
Loss after iteration 5000: 0.010027
Loss after iteration 6000: 0.009098
Loss after iteration 7000: 0.008321
Loss after iteration 8000: 0.007667
Loss after iteration 9000: 0.007112
Loss after iteration 10000: 0.006640
Loss after iteration 11000: 0.006236
Loss after iteration 12000: 0.005890
Loss after iteration 13000: 0.005592
Loss after iteration 14000: 0.005335
Loss after iteration 15000: 0.005112
Loss after iteration 16000: 0.004918
Loss after iteration 17000: 0.004749
Loss after iteration 18000: 0.004600
Loss after iteration 19000: 0.004470
Loss after iteration 20000: 0.004356
Loss after iteration 21000: 0.004254
Loss after iteration 22000: 0.004165
Loss after iteration 23000: 0.004085
Loss after iteration 24000: 0.004014
Loss after iteration 25000: 0.003951
Loss after iteration 26000: 0.003894
Loss after iteration 27000: 0.003843
Loss after iteration 28000: 0.003799
Loss after iteration 29000: 0.003762


In [12]:
output = feed_forward(model, X)
preds  = np.argmax(output[3] , axis=1)
preds

array([0, 0, 1, 1, 1], dtype=int64)